In [18]:
from pyclowderext import pyclowderext as pc_ext
import pandas as pd

# read in files
log = pd.read_csv('clowder_log_pdfs_20221214.csv')
source = pd.read_excel('PPRTV_NCEA_chemicals_downloaded.xlsx')

# define variables for API calls
userID = YOUR_USER_ID,
baseurl = 'https://clowder.edap-cluster.com',
apiKey = YOUR_API_KEY

In [19]:
# transform the source file into the metadata we'll be uploading

# don't mess with the files that didn't download in the last step!
source = source[source['downloaded'] == 1]
del source['downloaded']

# delete the index column that Pandas thinks I want. I shoulda done this in R
source = source.drop(source.columns[[0]], axis=1)

# sort everything alphabetically by URL to align it up correctly
source = source.sort_values('PPRTV Assessment')
log = log.sort_values('subDir1')

# match indices to avoid NaN outputs in new column (I shoulda done this in R!)
source.index = log.index

# add clowder IDs to the source file, as this is all the metadata we'll be uploading
source['clowder_id'] = log['uuid']
source['document_name'] = log['subDir1']

# Sort by chemical name just to tidy things up a bit
source = source.sort_values('Chemical')

In [59]:
for cid in source['clowder_id']:
    # remove the incorrect metadata I erroneously uploaded below
    pc_ext.remove_file_metadata(cid, 'https://clowder.edap-cluster.com', apiKey)
    # call the extractor to re-generate the hash metadata
    pc_ext.api_request(apiType = "POST",
               url = f'https://clowder.edap-cluster.com/api/files/{cid}/extractions',
               json = {'extractor': "ncsa.file.digest"},
               apiKey = apiKey)

...deleting metadata for file: 639a2f75e4b04f6bb14a1672
...deleting metadata for file: 639a2f75e4b04f6bb14a1674
...deleting metadata for file: 639a2f75e4b04f6bb14a1677
...deleting metadata for file: 639a2f75e4b04f6bb14a167a
...deleting metadata for file: 639a2f75e4b04f6bb14a167d
...deleting metadata for file: 639a2f75e4b04f6bb14a1680
...deleting metadata for file: 639a2f75e4b04f6bb14a1683
...deleting metadata for file: 639a2f75e4b04f6bb14a1686
...deleting metadata for file: 639a2f75e4b04f6bb14a1689
...deleting metadata for file: 639a2f75e4b04f6bb14a168c
...deleting metadata for file: 639a2f75e4b04f6bb14a168f
...deleting metadata for file: 639a2f75e4b04f6bb14a1692
...deleting metadata for file: 639a2f75e4b04f6bb14a1694
...deleting metadata for file: 639a2f75e4b04f6bb14a1696
...deleting metadata for file: 639a2f75e4b04f6bb14a1698
...deleting metadata for file: 639a2f75e4b04f6bb14a169a
...deleting metadata for file: 639a2f75e4b04f6bb14a169c
...deleting metadata for file: 639a2f75e4b04f6bb

...deleting metadata for file: 639a2faee4b04f6bb14a20f5
...deleting metadata for file: 639a2faee4b04f6bb14a20f7
...deleting metadata for file: 639a2faee4b04f6bb14a20f9
...deleting metadata for file: 639a2faee4b04f6bb14a20fb
...deleting metadata for file: 639a2faee4b04f6bb14a20fd
...deleting metadata for file: 639a2faee4b04f6bb14a20ff
...deleting metadata for file: 639a2faee4b04f6bb14a2101
...deleting metadata for file: 639a2faee4b04f6bb14a2103
...deleting metadata for file: 639a2faee4b04f6bb14a2105
...deleting metadata for file: 639a2faee4b04f6bb14a2107
...deleting metadata for file: 639a2faee4b04f6bb14a2109
...deleting metadata for file: 639a2faee4b04f6bb14a210b
...deleting metadata for file: 639a2faee4b04f6bb14a210d
...deleting metadata for file: 639a2faee4b04f6bb14a210f
...deleting metadata for file: 639a2faee4b04f6bb14a2111
...deleting metadata for file: 639a2faee4b04f6bb14a2113
...deleting metadata for file: 639a2faee4b04f6bb14a2115
...deleting metadata for file: 639a2faee4b04f6bb

In [60]:
# push the correct metadata
# for reasons unclear to me, it's not accepting my User ID and saying these data were uploaded by "Unknown"
# but they *are* being uploaded!
pc_ext.upload_file_metadata(metadata = source,
                           idCol = 'clowder_id',
                           userID = userID,
                           baseurl = 'https://clowder.edap-cluster.com',
                           apiKey = apiKey)

...pushing metadata to files...


In [20]:
# Export metadata for use as document map
source.to_excel('pprtv_ncea_document_map_12282021.xlsx', index = False)